In [2]:
import geopandas as gpd
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# open the shapefile
# gdf = gpd.read_file('/app/notebooks/rapprochements/Bordeaux/20240625 - ECHANTILLON BUILDING RNB/063_3_063_8_162_Building.shp')
gdf = gpd.read_file('/app/notebooks/rapprochements/Bordeaux/Bordeaux-tout/Bordeaux.shp')
gdf = gdf[['BUILDINGID', 'geometry']]
# print current crs
print(gdf.crs)

gdf = gdf.to_crs(epsg=4326)
gdf['geometry'] = gdf['geometry'].force_2d()
print(gdf.head())

/venv/lib/python3.12/site-packages/pyogrio/raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(


EPSG:3945
  BUILDINGID                                           geometry
0   28109912  POLYGON ((-0.73855 44.84228, -0.73853 44.84225...
1   28113973  POLYGON ((-0.65259 44.83922, -0.6526 44.83922,...
2   28118468  POLYGON ((-0.69782 44.83392, -0.69782 44.83392...
3   28100527  POLYGON ((-0.65619 44.84985, -0.6562 44.84984,...
4   28114929  POLYGON ((-0.66237 44.84621, -0.66238 44.84617...


In [4]:
# number of buildings
print(len(gdf))

274686


In [5]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection

# sample = gdf.sample(100)

# apply a function to each row, return a dataframe
def bordeaux_rnb_intersection(row):
    cursor = connection.cursor()
    geometry = row['geometry']
    sql = f"""with areas as (
select
	rnb_id,
	ST_AREA(ST_INTERSECTION(shape,
	ST_GeomFromText('{geometry}', 4326))) as area_intersection,
	ST_AREA(shape) as area_rnb,
	ST_AREA(ST_GeomFromText('{geometry}', 4326)) as area_bordeaux
from
	batid_building
where
	ST_Intersects(shape,
	ST_GeomFromText('{geometry}', 4326)))
        select
	{row['BUILDINGID']} as BUILDINGID, rnb_id, area_intersection / NULLIF(area_rnb, 0) as rnb_inclu_bdx, area_intersection / NULLIF(area_bordeaux, 0) as bdx_inclu_rnb
from
	areas"""

    cursor.execute(sql)

    results = cursor.fetchall()

    df = None
    if len(results) == 0:
        result = [{'BUILDINGID': row['BUILDINGID'], 'rnb_id': None, 'rnb_inclu_bdx': None, 'bdx_inclu_rnb': None}]
        df = pd.DataFrame(result)
    else:
        df = pd.DataFrame(results, columns=['BUILDINGID', 'rnb_id', 'rnb_inclu_bdx', 'bdx_inclu_rnb'])
    return df
    
# apply the function to each row
gdf_compute = gdf.apply(bordeaux_rnb_intersection, axis=1)
# concatenate the results
gdf_compute = pd.concat(gdf_compute.values)

# print(sample)

/tmp/ipykernel_1316/2008446435.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gdf_compute = pd.concat(gdf_compute.values)


In [6]:
gdf_compute.head()

,BUILDINGID,rnb_id,rnb_inclu_bdx,bdx_inclu_rnb
0,28109912,Y2PJP3T6D9SJ,1.000000,0.347379
0,28113973,PTXPN1FGPC1Y,0.999957,0.330095
1,28113973,GPYGZFHRAHS5,1.000000,0.094151
2,28113973,68K6H89MSEW8,0.995663,0.518983
0,28118468,None,NaN,NaN


In [7]:
# gdf_compute.to_csv('/app/notebooks/rapprochements/Bordeaux/intermediate_results.csv', index=False)
gdf_compute.to_csv('/app/notebooks/rapprochements/Bordeaux/intermediate_results_all.csv', index=False)

In [8]:
# gdf_compute = pd.read_csv('/app/notebooks/rapprochements/Bordeaux/intermediate_results.csv')
gdf_compute = pd.read_csv('/app/notebooks/rapprochements/Bordeaux/intermediate_results_all.csv')
gdf_compute.head()

,BUILDINGID,rnb_id,rnb_inclu_bdx,bdx_inclu_rnb
0,28109912,Y2PJP3T6D9SJ,1.000000,0.347379
1,28113973,PTXPN1FGPC1Y,0.999957,0.330095
2,28113973,GPYGZFHRAHS5,1.000000,0.094151
3,28113973,68K6H89MSEW8,0.995663,0.518983
4,28118468,NaN,NaN,NaN


In [9]:
gdf_compute_2 = gdf_compute.copy()

# a match is when both intersection ratios are above 0.8
gdf_compute_2['match_1_1'] = (gdf_compute_2['rnb_inclu_bdx'] > 0.8) & (gdf_compute_2['bdx_inclu_rnb'] > 0.8)
gdf_compute_2['rnb_inclu_bdx_b'] = (gdf_compute_2['rnb_inclu_bdx'] > 0.8) & (gdf_compute_2['bdx_inclu_rnb'] <= 0.8)
gdf_compute_2['bdx_inclu_rnb_b'] = (gdf_compute_2['bdx_inclu_rnb'] > 0.8) & (gdf_compute_2['rnb_inclu_bdx'] <= 0.8)
gdf_compute_2['small_intersection'] = (gdf_compute_2['rnb_inclu_bdx'] <= 0.8) & (gdf_compute_2['bdx_inclu_rnb'] <= 0.8)

# print(gdf_compute_2.head(10))

# print gdf_compute_2 for the ID 6318700
# print(gdf_compute_2[gdf_compute_2['BUILDINGID'] == 6318700])

intersections = gdf_compute_2[gdf_compute_2['rnb_id'].notnull()].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
match_1_1 = gdf_compute_2[gdf_compute_2['match_1_1']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
rnb_inclu_bdx = gdf_compute_2[gdf_compute_2['rnb_inclu_bdx_b']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
bdx_inclu_rnb = gdf_compute_2[gdf_compute_2['bdx_inclu_rnb_b']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
small_intersection = gdf_compute_2[gdf_compute_2['small_intersection']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()

# merge all the dataframes
df_merge = pd.DataFrame(gdf_compute['BUILDINGID'].unique(), columns=['BUILDINGID'])
df_merge = df_merge.merge(intersections, on='BUILDINGID', how='left', suffixes=('', '_intersections'))
df_merge = df_merge.merge(match_1_1, on='BUILDINGID', how='left', suffixes=('', '_match_1_1'))
df_merge = df_merge.merge(rnb_inclu_bdx, on='BUILDINGID', how='left', suffixes=('', '_rnb_inclu_bdx'))
df_merge = df_merge.merge(bdx_inclu_rnb, on='BUILDINGID', how='left', suffixes=('', '_bdx_inclu_rnb'))
df_merge = df_merge.merge(small_intersection, on='BUILDINGID', how='left', suffixes=('', '_small_intersection'))


# # clean None values in lists
# df_merge['rnb_id'] = df_merge['rnb_id'].apply(lambda l: [rnb_id for rnb_id in l if rnb_id is not None])


df_merge.head(20)


,BUILDINGID,rnb_id,rnb_id_match_1_1,rnb_id_rnb_inclu_bdx,rnb_id_bdx_inclu_rnb,rnb_id_small_intersection
0,28109912,[Y2PJP3T6D9SJ],NaN,[Y2PJP3T6D9SJ],NaN,NaN
1,28113973,"[PTXPN1FGPC1Y, GPYGZFHRAHS5, 68K6H89MSEW8]",NaN,"[PTXPN1FGPC1Y, GPYGZFHRAHS5, 68K6H89MSEW8]",NaN,NaN
2,28118468,NaN,NaN,NaN,NaN,NaN
3,28100527,"[MYSHX1RJT8EJ, VG6AVFQF9FSA, 544P3RY978C8]",NaN,"[MYSHX1RJT8EJ, VG6AVFQF9FSA, 544P3RY978C8]",NaN,NaN
4,28114929,"[CNSQT2AMVFNG, 5FTWFC56S579]",NaN,[5FTWFC56S579],NaN,[CNSQT2AMVFNG]
5,28100000,"[WFPTM13MPPM8, QVKGY9NVTD7N]",NaN,"[WFPTM13MPPM8, QVKGY9NVTD7N]",NaN,NaN
6,28100001,[QDME34HYNJG8],[QDME34HYNJG8],NaN,NaN,NaN
7,28100002,"[8W274KYCQXVX, SPFEBGPVEXMR]",NaN,"[8W274KYCQXVX, SPFEBGPVEXMR]",NaN,NaN
8,28100005,[JNM9R6V33889],NaN,[JNM9R6V33889],NaN,NaN
9,28100006,NaN,NaN,NaN,NaN,NaN


In [42]:
# print number of rows
N = len(df_merge)
print("Nombre de bâtiments dans l'échantillon")
print(N)

print("Nombre de bâtiments sans intersection")
n = len(df_merge[df_merge['rnb_id'].isnull()])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments avec intersection")
n = len(df_merge[df_merge['rnb_id'].notnull()])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments avec match 1:1")
n = len(df_merge[df_merge['rnb_id_match_1_1'].notnull()])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments qui contiennet un ou plusieurs bâtiments RNB inclus")
n = len(df_merge[df_merge['rnb_id_rnb_inclu_bdx'].notnull() & df_merge['rnb_id_match_1_1'].isnull()])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments qui sont inclus dans un bâtiment RNB")
n = len(df_merge[df_merge['rnb_id_bdx_inclu_rnb'].notnull() & df_merge['rnb_id_match_1_1'].isnull()])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments avec uniquement des petites intersections")
n = len(df_merge[df_merge['rnb_id_small_intersection'].notnull() & df_merge['rnb_id_match_1_1'].isnull() & df_merge['rnb_id_rnb_inclu_bdx'].isnull() & df_merge['rnb_id_bdx_inclu_rnb'].isnull()])
print(n)
print(f"soit {round(n/N*100, 1)}%")



Nombre de bâtiments dans l'échantillon
274667
Nombre de bâtiments sans intersection
29304
soit 10.7%
Nombre de bâtiments avec intersection
245363
soit 89.3%
Nombre de bâtiments avec match 1:1
91142
soit 33.2%
Nombre de bâtiments qui contiennet un ou plusieurs bâtiments RNB inclus
123853
soit 45.1%
Nombre de bâtiments qui sont inclus dans un bâtiment RNB
8503
soit 3.1%
Nombre de bâtiments avec uniquement des petites intersections
21741
soit 7.9%


In [11]:
# df_merge.to_csv('/app/notebooks/rapprochements/Bordeaux/results_echantillon.csv', index=False)
df_merge.to_csv('/app/notebooks/rapprochements/Bordeaux/results_all.csv', index=False)


In [58]:
# filter df with len of rnb_id_rnb_inclu_bdx > 1
df_merge_filtered = df_merge[['BUILDINGID', 'rnb_id_rnb_inclu_bdx']]
df_merge_filtered = df_merge_filtered[df_merge_filtered['rnb_id_rnb_inclu_bdx'].notnull()]
df_merge_filtered = df_merge_filtered[df_merge_filtered['rnb_id_rnb_inclu_bdx'].apply(lambda l: len(l) > 1)]
print(df_merge_filtered.head())
building_ids = df_merge_filtered['BUILDINGID']
print(len(building_ids))

    BUILDINGID                               rnb_id_rnb_inclu_bdx
1     28113973         [PTXPN1FGPC1Y, GPYGZFHRAHS5, 68K6H89MSEW8]
3     28100527         [MYSHX1RJT8EJ, VG6AVFQF9FSA, 544P3RY978C8]
5     28100000                       [WFPTM13MPPM8, QVKGY9NVTD7N]
7     28100002                       [8W274KYCQXVX, SPFEBGPVEXMR]
13    28100010  [AJ6FSH49KAMW, CVGYJA2T6HZE, 3FAMPZP74Z79, VCD...
50299


In [59]:
building_ids = building_ids.astype(str)
# pad with 0 to have 8 digits
building_ids = building_ids.str.zfill(8)

gdf_filtered = gdf[gdf['BUILDINGID'].isin(building_ids)]
print(gdf_filtered.head())
# write to file
gdf_filtered.to_file('/app/notebooks/rapprochements/Bordeaux/buildings_sous_decoupes.shp')

   BUILDINGID                                           geometry
1    28113973  POLYGON ((-0.65259 44.83922, -0.6526 44.83922,...
3    28100527  POLYGON ((-0.65619 44.84985, -0.6562 44.84984,...
5    28100000  POLYGON ((-0.68489 44.83838, -0.6849 44.83838,...
7    28100002  POLYGON ((-0.65173 44.83282, -0.65173 44.83282...
13   28100010  POLYGON ((-0.6265 44.81846, -0.62654 44.81845,...


In [60]:
print(len(gdf_filtered))

50316
